In [ ]:
import pandas as pd
import numpy as np

# best_model_strength (GPR)
# best_model_elongation (GPR or XGBoost)
# best_model_healing (Random Forest)

class Smart_DAPU_Designer:
    def __init__(self, model_str, model_elo, model_heal):
        self.model_str = model_str
        self.model_elo = model_elo
        self.model_heal = model_heal
        
    def predict_all(self, X_virtual):
        """
        输入虚拟配方 X_virtual (DataFrame)，输出三个性能的预测表
        """
        # 1. 预测
        pred_str = self.model_str.predict(X_virtual)
        pred_elo = self.model_elo.predict(X_virtual)
        pred_heal = self.model_heal.predict(X_virtual)
        
        # 2. 打包结果
        results = X_virtual.copy()
        results['Pred_Strength'] = pred_str
        results['Pred_Elongation'] = pred_elo
        results['Pred_Healing'] = pred_heal
        
        return results

    def screen_recipes(self, results_df, strategy='High_Strength'):
        """
        根据策略筛选配方
        """
        if strategy == 'High_Strength':
            # 策略：强度优先，愈合随缘
            # 筛选条件：强度高，且伸长率不要太离谱(>100%)
            candidates = results_df[
                (results_df['Pred_Strength'] > 35) & 
                (results_df['Pred_Elongation'] > 100) # 宽松的约束
            ].sort_values(by='Pred_Strength', ascending=False)
            
        elif strategy == 'High_Healing':
            # 策略：愈合优先 
            # 筛选条件：愈合好，且强度不能太烂(>5MPa)
            candidates = results_df[
                (results_df['Pred_Healing'] > 80) & 
                (results_df['Pred_Strength'] > 5)
            ].sort_values(by='Pred_Healing', ascending=False)
            
        return candidates.head(5)

# --- 执行操作 ---
# 1. 生成虚拟配方库 (Virtual Library)
# X_virtual = ... (你的生成代码)

# 2. 初始化设计器
designer = Smart_DAPU_Designer(best_model_strength, best_model_elongation, best_model_healing)

# 3. 全局扫描
all_predictions = designer.predict_all(X_virtual)

# 4. 挑选出你要做的两个“极端”样品
target_A = designer.screen_recipes(all_predictions, strategy='High_Strength')
target_B = designer.screen_recipes(all_predictions, strategy='High_Healing')

print("=== 目标 A: 刚性验证配方 ===")
print(target_A[['X1', 'X2', 'X3', 'Pred_Strength', 'Pred_Healing']])

print("\n=== 目标 B: 愈合验证配方 ===")
print(target_B[['X1', 'X2', 'X3', 'Pred_Strength', 'Pred_Healing']])